In [ ]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os

In [ ]:
#set up logging 
logging.basicConfig(level=logging.info)

debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

In [15]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [ ]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        debug(obj['Key'])
        debug(obj['Size'])
else:
    error("No objects found in the bucket.")

In [32]:
# specify the folder path (in the current working directory) for your object 
# if the folder doesn't exist, create it
save_path = os.path.join(os.getcwd(), r'junk')
if not os.path.exists(save_path):
    os.makedirs(save_path)

# create new text file within the folder
dest_file_path = os.path.join(save_path, r'test.txt')

# write data to the new text file
with open(dest_file_path, 'w') as fh:
    fh.write('example example example\n')

In [ ]:
# put the new text file into your object storage bucket
s3_client.put_object(Bucket=bucket, Key=dest_file_path)

In [34]:
# define the function for creating presigned URL 
# running the function will return a shareable URL in the print output
def create_presigned_url(
    bucket_name = bucket,
    object_name = dest_file_path,
    expiration=180,
):

    try:
        response = s3_client.generate_presigned_url(
            "get_object",
            Params={"Bucket": bucket_name, "Key": object_name},
            ExpiresIn=expiration,
            )
        print(response)

    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [ ]:
# run the presigned URL function with the given parameters
create_presigned_url(bucket, dest_file_path)

In [ ]:
# delete the remote object
s3_client.delete_object(
    Bucket=bucket, 
    Key=dest_file_path
    )